# News Headlines Sentiment

Use the news api to pull the latest news articles for bitcoin and ethereum and create a DataFrame of sentiment scores for each coin. 

Use descriptive statistics to answer the following questions:
1. Which coin had the highest mean positive score?
2. Which coin had the highest negative score?
3. Which coin had the highest positive score?

In [ ]:
# Initial imports
import os
import pandas as pd
from newsapi import NewsApiClient
from dotenv import load_dotenv
from pathlib import Path
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
loan_dotenv()
analyzer = SentimentIntensityAnalyzer()

%matplotlib inline

In [ ]:
# Download/Update the VADER Lexicon
nltk.download('vader_lexicon')

# Initialize the VADER sentiment analyzer
analyzer = SentimentIntensityAnalyzer()


In [ ]:
# Read your api key environment variable
api_key = os.getenv("news_api")


In [ ]:
# Create a newsapi client
newsapi = NewsApiClient(api_key=api_key)

In [ ]:
# Fetch the Bitcoin news articles
bitcoin_headlines = newsapi.get_everything(
    q="bitcoin", language="en",page_size=100, sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Bitcoin: {bitcoin_headlines['totalResults']}")

# Show sample article
bitcoin_headlines["articles"][0]

In [ ]:
bitcoin_df = pd.DataFrame.from_dict(bitcoin_headlines['articles'])
bitcoin_df.head()

In [ ]:
file_path = Path("../Data/")
bitcoin_df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [ ]:
file_path = Path("../Data/")
bitcoin_df.to_pickle(file_path)

In [ ]:
# Fetch the Ethereum news articles
ethereum_headlines = newsapi.get_everything(
    q="ethereum and Ethereum", language="en",page_size=100, sort_by="relevancy"
)

# Print total articles
print(f"Total articles about Ethereum: {ethereum_headlines['totalResults']}")

# Show sample article
ethereum_headlines["articles"][0]


In [ ]:
# Create the Bitcoin sentiment scores DataFrame
# Create the Bitcoin sentiment scores DataFrame
bitcoin_sentiment = []

for article in bitcoin_headlines["articles"]:
    try:
        
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        neg = sentiment["neg"]
        neu = sentiment["neu"]
        pos = sentiment["pos"]
        
        bitcoin_sentiments.append({
            "Compound": compound,
            "Negative": neg,
            "Neutral": neu,
            "Positive": pos,
            "Text": text,
 
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
bitcoin_sentiment_df = pd.DataFrame(bitcoin_sentiment)

# Reorder DataFrame columns
cols = ["Compound", "Negative", "Neutral", "Positive", "Text"]
bitcoin_sentiment_df = bitcoin_sentiment_df[cols]

bitcoin_sentiment_df.head()

In [ ]:
file_path = Path("../Data/")
bitcoin_sentiment_df.to_csv(file_path, index=False, encoding='utf-8-sig')

In [ ]:
file_path = Path("../Data/")
bitcoin_sentiment_df.to_pickle(file_path)

In [ ]:
# Create the ethereum sentiment scores DataFrame
# Create the ethereum sentiment scores DataFrame
ethereum_sentiments = []

for article in ethereum_headlines["articles"]:
    try:
        
        text = article["content"]
        sentiment = analyzer.polarity_scores(text)
        compound = sentiment["compound"]
        neg = sentiment["neg"]
        neu = sentiment["neu"]
        pos = sentiment["pos"]
        
        ethereum_sentiments.append({
            "Compound": compound,
            "Negative": neg,
            "Neutral": neu,
            "Positive": pos,
            "Text": text,
 
        })
        
    except AttributeError:
        pass
    
# Create DataFrame
ethereum_sentiments_df = pd.DataFrame(ethereum_sentiments)

# Reorder DataFrame columns
cols = ["Compound", "Negative", "Neutral", "Positive", "Text"]
ethereum_sentiments_df = ethereum_sentiments_df[cols]

ethereum_sentiments_df.head()

In [ ]:
# Describe the Bitcoin Sentiment
bitcoin_sentiments_df.describe()

In [ ]:
# Describe the Ethereum Sentiment
ethereum_sentiments_df.describe()

### Questions:

Q: Which coin had the highest mean positive score?

A: Bitcoin

Q: Which coin had the highest compound score?

A: Bitcoin

Q. Which coin had the highest positive score?

A: Bitcoin


---

# Tokenizer

In this section, you will use NLTK and Python to tokenize the text for each coin. Be sure to:
1. Lowercase each word
2. Remove Punctuation
3. Remove Stopwords

In [ ]:
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from string import punctuation
import re

In [ ]:
# Code to download wordnet corpora
import nltk
nltk.download('stopwords')

lemmatizer = WordNetLemmatizer()

In [ ]:
def tokenizer(text):
    """Tokenizes text."""
# Remove the punctuation
regex = re.compile("[^a-zA-Z ]")
clean_text = regex.sub('', text)
    
# Create a list of the words
sentence = sent_tokenize(clean_text)
words = []
for sent in sentence:
    word = word_tokenize(sent)
    words = words + word    
        
# Convert the words to lowercase
words_lc = [word.lower() for word in words]  


# Remove the stop words
sw = set(stopwords.words('english'))
words_sw = [word for word in words_lc if word not in sw]



# Expand the default stopwords list if necessary


In [ ]:
# Complete the tokenizer function
def tokenizer(text):
    """Tokenizes text."""
    
    # Create a list of the words
    # Convert the words to lowercase 
    # Remove the punctuation  
    # Remove the stop words 
    # Lemmatize Words into root words
     sw= set(stopwords.words('english'))
    regex= re.compile("[^a-zA-Z ]")
    
    re_clean = regex.sub('', str(text))
    words= word_tokenize(re_clean)
    lem=[lemmatizer.lemmatize(word) for word in words]
    tokens= [word.lower() for word in lem if word.lower() not in sw ]
    
    return tokens


In [ ]:
# Create a new tokens column for bitcoin
Bitcoin_df["tokens"] = Bitcoin_df.Text.apply(tokenizer)
Bitcoin_df.head()

In [ ]:
# Create a new tokens column for ethereum
Ethereum_df["tokens"] = Ethereum_df.Text.apply(tokenizer)
Ethereum_df.head()

---

# NGrams and Frequency Analysis

In this section you will look at the ngrams and word frequency for each coin. 

1. Use NLTK to produce the n-grams for N = 2. 
2. List the top 10 words for each coin. 

In [ ]:
from collections import Counter
from nltk import ngrams

In [ ]:
# Generate the Bitcoin N-grams where N=2
N=2
bigram_counts_B = Counter(ngrams(bitcoin_p, N))
print(dict(bigram_counts_B))

In [ ]:
# Generate the Ethereum N-grams where N=2
N= 2
bigram_counts_E = Counter(ngrams(ethereum_p, N))
print(dict(bigram_counts_E))

In [ ]:
# Use the token_count function to generate the top 10 words from each coin
def token_count(tokens, N=10):
    """Returns the top N tokens from the frequency count"""
    return Counter(tokens).most_common(N)

In [ ]:
# Get the top 10 words for Bitcoin
bitcoin_common= token_count(bitcoin_p, 10)
bitcoin_common

In [ ]:
# Get the top 10 words for Ethereum
ethereum_common= token_count(ethereum_p,10)
ethereum_common

# Word Clouds

In this section, you will generate word clouds for each coin to summarize the news for each coin

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [20.0, 10.0]

In [ ]:
# Generate the Bitcoin word cloud
cloud1 = WordCloud(background_color='grey').generate(Bitcoin_df.Text.str.cat())
cloud1
plt.axis("off")
plt.imshow(cloud1)

In [ ]:
# Generate the Ethereum word cloud
cloud2 = WordCloud(background_color='grey').generate(Ethereum_df.Text.str.cat())
cloud2
plt.axis("off")
plt.imshow(cloud2)

# Named Entity Recognition

In this section, you will build a named entity recognition model for both coins and visualize the tags using SpaCy.

In [ ]:
import spacy
from spacy import displacy

In [ ]:
# Optional - download a language model for SpaCy
# !python -m spacy download en_core_web_sm

In [ ]:
# Load the spaCy model
nlp = spacy.load('en_core_web_sm')

## Bitcoin NER

In [ ]:
# Concatenate all of the bitcoin text together
all_concat_bitcoin = Bitcoin_df.Text.str.cat()
all_concat_bitcoin

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
bitcoin_doc = nlp(all_concat_bitcoin)
bitcoin_doc
bitcoin_doc.user_data["Title"] = "Bitcoin NER"

In [ ]:
# Render the visualization
displacy.render(bitcoin_doc, style = 'ent')

In [ ]:
# List all Entities
for i in bitcoin_doc.ents:
    print(i.text, i.label_)

---

## Ethereum NER

In [ ]:
# Concatenate all of the bitcoin text together
all_concat_ethereum = Ethereum_df.Text.str.cat()
all_concat_ethereum

In [ ]:
# Run the NER processor on all of the text
# YOUR CODE HERE!

# Add a title to the document
ethereum_doc = nlp(all_concat_ethereum)
ethereum_doc
ethereum_doc.user_data["Title"] = "Ethereum NER"

In [ ]:
# Render the visualization
displacy.render(ethereum_doc, style = 'ent')

In [ ]:
# List all Entities
for i in ethereum_doc.ents:
    print(i.text, i.label_)